In [2]:
import re
import openai
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path = ".env")
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

### Rewrite for language

In [ ]:
prompt = """Your task is to revise a section of an academic paper (in LaTeX format) based on the review advise. Follow these steps:

I will provide you with a section (titled "{section_label}") of the paper. You need to check the section for:

Language: Look for grammatical errors, colloquial expressions, or any subpar phrases and suggest improvements.
Logical Flow: Ascertain whether the content follows a reasonable sequence, whether transitions between ideas are smooth, and pinpoint any incoherence in the logic.
Both the whole paper and individual sections' logical flow are depicted using JSON-encoded directed acyclic graph (DAG) structures.

The logical structure of the entire paper is: {paper_structure}

The logical structure of the specific section is: {section_structure}

The content of the section is: {section_content}

Formatting Requirements:

Present your output in Markdown format. Convert LaTeX section content to Markdown, omitting images, tables, and other non-textual elements. Divide your output into "Section Content with Annotations" and "Logical Flow Commentary."
"Section Content with Annotations": Wrap any sentences with language issues in "@" symbols and immediately follow them with a parenthesis stating what the issue is.
"Logical Flow Commentary": After presenting the annotated section, add a separate part summarizing the logical flow of the paragraph. State whether it's logical or if any issues are present, and describe them briefly.
Please focus solely on the language-related logical flow issues for this task, without considering the factual accuracy of the content.

Here's an example for reference:

{content_analysis_example}

Ensure your response is clear and it enables gpt to understand and fulfill the task effectively."""


In [45]:
example = {'section_content': """ The Urban Life and Air Pollution task at MediaEval 2022 required participants to predict the air quality index (AQI) value at +1, +5 and +7 days using an archive of air quality, weather and images from 16 CCTV cameras, one image taken every 60 seconds \cite{UA22}. Participating groups were required to download the data from online sources for local processing.
Gaps in air quality datasets are common with the problem exacerbated for data gathered in poorer or developing countries \cite{PINDER2019116794, Falge2001, Hui2004, Moffat2007, Kim2020}. <span style="color:red;">In this paper we describe how we addressed the very large gaps in data that we encountered in the data we downloaded.</span>(<span style="color:orange;">This sentence is repetitive with the use of "data" three times. It could be rephrased for clarity and conciseness.</span>)""", 
           'section_label': 'Introduction',
           'revise_result': """The Urban Life and Air Pollution task at MediaEval 2022 required participants to predict the air quality index (AQI) value at +1, +5, and +7 days using an archive of air quality, weather, and images from 16 CCTV cameras, one image taken every 60 seconds \cite{UA22}. Participating groups were required to download the data from online sources for local processing.
Gaps in air quality datasets are common, with the problem exacerbated for data gathered in poorer or developing countries \cite{PINDER2019116794, Falge2001, Hui2004, Moffat2007, Kim2020}. In this paper, we detail our approach to addressing the substantial gaps encountered within the downloaded datasets."""}

In [ ]:
在这段文本中，已经标注好了具体的language issues，其中红色的部分是需要修改的内容（用<span style="color:red;">和</span>warp起来），橙色的部分是对修改内容的解释（用<span style="color:orange;">和</span>warp起来）。
你需要根据这些标注，修改这段文本，使得它更加符合语法规范，也要保证修改后的文本与原文的意思一致。同时对于不需要修改的部分，要保证不改变原本的内容，要一字不差的保留下来。

In [46]:
language_issue_prompt = """"Your task is to revise a section of an academic paper (in LaTeX format) based on the review advise. Follow these requirements: 

I will provide you with a section (titled "{section_label}") of the paper. In this text, specific language issues have already been marked. The parts needing revision are wrapped with "@" symbols and immediately follow a parenthesis stating what the issue is.
Based on these annotations, the task is to modify the text to make it more grammatically correct, ensuring that the meaning of the revised text remains consistent with the original. 
It is also crucial to retain the unmodified parts of the text exactly as they are, without any alterations.

The content of the section is: {section_content}

Here's an example for reference:

provided section: {example_content}

revised result: {revise_result}

Ensure your response fully complies with compliance requirements and fulfill the task effectively."""

In [34]:
language_issue_prompt_zero_shot = """"Your task is to revise a section of an academic paper (in LaTeX format) based on the review advise. Follow these requirements: 

I will provide you with a section (titled "{section_label}") of the paper in Latex format. In this text, specific language issues have already been marked. The parts needing revision are indicated in red, wrapped with <span style="color:red;"> and </span>. The explanations for these revisions are in orange, wrapped with <span style="color:orange;"> and </span>. 
Based on these annotations, the task is to modify the text to make it more grammatically correct, ensuring that the meaning of the revised text remains consistent with the original. 
It is also crucial to retain the unmodified parts of the text exactly as they are, without any alterations.

The content of the section is: {section_content}

Formatting Requirements:

Present your output in Latex format. You should only output the revised text directly without anything additional.

Ensure your response fully complies with compliance requirements and fulfill the task effectively."""

In [49]:
test = """\\section\{method\}\n Planning a trip to the moon differs in no essential respect from planning a trip to the beach. \
@You hve to decide what to take along, what to leave behind.@(There is a typographical error: "hve" should be "have.". It is generally more common to see "take with you" instead of "take along.") \
Should the thermos jug go? The child's rubber horse? The dill pickles? \
@These are the sometimes fateful decisions on which the success or failure of the whole outing turns.@\
(The first issue with this sentence is the subject-verb disagreement. "These" is plural, but "is" is singular. \
The correct verb form should be "are" to match the plural subject.The last part of the sentence, "on which the success or failure of the whole outing turns," \
is a bit awkward. A smoother way to write this might be: "These decisions can sometimes be crucial, determining the success or failure of the entire outing.")
Among the items they sent along, of course, was the little jointed flagpoles and the flag that could be stiffened to the breeze that did not blow. \
(It is traditional among explorers to plant the flag.) Yet the two men who stepped out on the surface of the moon were in a class by themselves and should \
have been equipped accordingly: they were of the new breed of men, those who had seen the earth whole."""

In [50]:
prompt = language_issue_prompt.format(section_content = test, section_label = example["section_label"], example_content = example["section_content"], revise_result = example["revise_result"])

In [25]:
from llm_api import llm_request

In [51]:
response = llm_request(prompt)

In [52]:
print(response)

\section{method}
Planning a trip to the moon differs in no essential respect from planning a trip to the beach. You have to decide what to take with you, what to leave behind. Should the thermos jug go? The child's rubber horse? The dill pickles? These decisions can sometimes be crucial, determining the success or failure of the entire outing. Among the items they sent along, of course, were the little jointed flagpoles and the flag that could be stiffened to the breeze that did not blow. (It is traditional among explorers to plant the flag.) Yet the two men who stepped out on the surface of the moon were in a class by themselves and should have been equipped accordingly: they were of the new breed of men, those who had seen the earth whole.


In [44]:
print(test)

Planning a trip to the moon differs in no essential respect from planning a trip to the beach. @You hve to decide what to take along, what to leave behind.@(There is a typographical error: "hve" should be "have.". It is generally more common to see "take with you" instead of "take along.") Should the thermos jug go? The child's rubber horse? The dill pickles? @These are the sometimes fateful decisions on which the success or failure of the whole outing turns.@(The first issue with this sentence is the subject-verb disagreement. "These" is plural, but "is" is singular. The correct verb form should be "are" to match the plural subject.The last part of the sentence, "on which the success or failure of the whole outing turns," is a bit awkward. A smoother way to write this might be: "These decisions can sometimes be crucial, determining the success or failure of the entire outing.")
Among the items they sent along, of course, was the little jointed flagpoles and the flag that could be stif

### Rewrite for logic

In [45]:
section_structure = {'nodes': [
                      {'name': 'Challenge Overview',
                       'content': 'The Urban Life and Air Pollution task at MediaEval 2022 is introduced, which required participants to predict the air quality index (AQI) value at future intervals using a variety of data sources.',
                       'parents': []}, {'name': 'Data Gaps Issue',
                                        'content': 'The paper acknowledges the common issue of gaps in air quality datasets, which is particularly problematic in poorer or developing countries.',
                                        'parents': ['Challenge Overview']}, {'name': 'Research Contribution',
                                                                             'content': 'The paper outlines its contribution by describing the approach taken to address the large gaps in the air quality data encountered.',
                                                                             'parents': ['Data Gaps Issue']}], 'edges': [
                      {'from': 'Challenge Overview', 'to': 'Data Gaps Issue'}, {'from': 'Data Gaps Issue', 'to': 'Research Contribution'}]}
example_content = """\\label={sec:intro}

\\begin{comment}
According to the World Health Organisation (WHO), 91\% of the world's population reside in conditions where WHO's air quality guidelines levels were not met \cite{organizacion2021global}. This report on 2016 also showed that ambient (outdoor) air pollution in both cities and rural areas was estimated to cause 4.2 million premature deaths worldwide. The research concluded that policies and investments supporting cleaner transport, energy-efficient homes, power generation, industry and better municipal waste management would would be crucial to the reduction of outdoor air pollution. In a separate report, it is estimated that air pollution globally accounts for roughly seven million premature deaths a year \cite{Gar21}, where it was again stated that the majority of those deaths are caused by outdoor air pollution with the rest generally attributed to poor air quality from indoor cooking. While the majority of these deaths occur in developing countries, with China and India accounting for roughly 50\%, developed countries also have a problem with deaths resulting from air pollution.
In this research, the focus will mostly be on the modelling of concentrations in particulate matter - tiny particles in the air generated both by natural processes and human activity. These particles are generally 
12  categorised (in the public health domain) by their diameter; fine particles with diameter less than 2.5 $\mu$m are referred to as "PM2.5" and coarse particles with diameter between 2.5 and 10 $\mu$m are referred to as "PM10".\\
\\end{comment}

The Urban Life and Air Pollution task at MediaEval 2022 required participants to predict the air quality index (AQI) value at +1, +5 and +7 days using an archive of air quality, weather and images from 16 CCTV cameras, one image taken every 60 seconds  \cite{UA22}. Participating groups were required to download the data from online sources for local processing.
Gaps in air quality datasets are common with the problem exacerbated for data gathered in poorer or developing countries \cite{PINDER2019116794, Falge2001, Hui2004, Moffat2007, Kim2020}. In this paper we describe how we addressed the very large gaps in data that we encountered in the data we downloaded."""

example_feedback = """The logical flow of the Introduction section is clear and follows a reasonable sequence. It begins with an overview of the challenge that sets the context for the research. It then transitions to discussing a specific issue related to the challenge, namely the gaps in air quality datasets, which is particularly problematic in poorer or developing countries. Finally, it outlines the paper's contribution by stating that the approach taken to address the large gaps in the air quality data will be described. The transitions between ideas are smooth, and the content is coherent. However, the last sentence could be improved to avoid repetition and enhance clarity."""

example_result = """\\label={sec:intro}
\\begin{comment}
According to the World Health Organisation (WHO), 91\% of the world's population reside in conditions where WHO's air quality guidelines levels were not met \cite{organizacion2021global}. This report from 2016 also indicated that ambient (outdoor) air pollution in both cities and rural areas was estimated to cause 4.2 million premature deaths worldwide. The research emphasized that implementing policies and investments to support cleaner transport, energy-efficient homes, power generation, industry, and better municipal waste management could significantly reduce outdoor air pollution. Another report estimates that air pollution globally accounts for approximately seven million premature deaths annually \cite{Gar21}, with the majority of these deaths being attributed to outdoor air pollution and the remainder largely due to poor indoor air quality from cooking practices. While most of these deaths occur in developing countries, with China and India contributing to about 50\%, developed nations are not exempt from the detrimental effects of air pollution.
In this research, we primarily examine the modelling of particulate matter concentrations—tiny particles in the air produced by both natural processes and human activities. These particles are typically classified by their diameter; fine particles with a diameter of less than 2.5 $\mu$m are denoted as "PM2.5," and coarse particles with a diameter between 2.5 and 10 $\mu$m are labeled as "PM10".
\\end{comment}

The Urban Life and Air Pollution task at MediaEval 2022 challenged participants to forecast the air quality index (AQI) for future intervals using a diverse set of data, including air quality metrics, weather conditions, and visual data from 16 CCTV cameras capturing images every minute \cite{UA22}. Participants were tasked with retrieving this data from various online repositories for subsequent local analysis.
Data gaps are a prevalent issue in air quality datasets, with this challenge being more pronounced in data collected from less affluent or developing regions \cite{PINDER2019116794, Falge2001, Hui2004, Moffat2007, Kim2020}. In this paper, we delineate our methodology for addressing the substantial data voids we encountered within the datasets we utilized.
"""

example = {"example_review" : example_feedback, "section_content": example_content, "section_label": "Introduction",
           "example_structure": section_structure, "example_result": example_result}


In [27]:
prompt = """Your task is to revise a section of an academic paper (in LaTeX format) based on the review advise. Follow these requirements:

I will provide you with a section (titled "{section_label}") of the paper in Latex format. 
Additionally, I will supply a review of this document segment. The review will critique the logical flow of the text. 
Based on this review, your task is to modify the text to enhance its logical coherence while preserving its core message. 
The modification should adhere to two criteria: (1) fidelity, ensuring the revised text remains true to the original intent and content, and (2) improved logical flow, making the content more academically sound, clear, and engaging.

The content of the section is: {section_content}

The logical structure of the specific section is: {section_structure}

The review advise is: {review_advise}
        
Here's an example for reference:

provided section: {example_content}

logical structure: {example_structure}

review advise: {example_review}

revised result: {revise_result}

Formatting Requirements:

Present your output in Latex format aligned with provided section. You should only output the revised text directly without anything additional.

Ensure your response fully complies with compliance requirements and fulfill the task effectively."""

In [40]:
prompt_zero = """Your task is to revise a section of an academic paper (in LaTeX format) based on the review advise. Follow these requirements:

I will provide you with a section (titled "{section_label}") of the paper in Latex format. 
Additionally, I will supply a review of this document segment. The review will critique the logical flow of the text. 
Based on this review, your task is to modify the text to enhance its logical coherence while preserving its core message. 
The modification should adhere to three criteria: 
(1) fidelity, ensuring the revised text remains true to the original intent and content, and 
(2) improved logical flow, making the content more academically sound, clear, and engaging, and
(3) integrity, ensuring that no information loss occurs, and maintaining the integrity of the original information as much as possible

The content of the section is: {section_content}

The logical structure of the specific section is: {section_structure}

The review advise is: {review_advise}
        
Formatting Requirements:

Present your output in Latex format aligned with provided section. You should only output the revised text directly without anything additional.

Ensure your response fully complies with compliance requirements and fulfill the task effectively."""

In [41]:
prompt = prompt_zero.format(section_label=example['section_label'], section_content=example['section_content'], section_structure=example['example_structure'], review_advise=example['example_review'])

In [42]:
print(prompt)

Your task is to revise a section of an academic paper (in LaTeX format) based on the review advise. Follow these requirements:

I will provide you with a section (titled "Introduction") of the paper in Latex format. 
Additionally, I will supply a review of this document segment. The review will critique the logical flow of the text. 
Based on this review, your task is to modify the text to enhance its logical coherence while preserving its core message. 
The modification should adhere to two criteria: 
(1) fidelity, ensuring the revised text remains true to the original intent and content, and 
(2) improved logical flow, making the content more academically sound, clear, and engaging, and
(3) integrity, ensuring that no information loss occurs, and maintaining the integrity of the original information as much as possible

The content of the section is: \label={sec:intro}

\begin{comment}
According to the World Health Organisation (WHO), 91\% of the world's population reside in conditio

In [43]:
from llm_api import llm_request
response = llm_request(prompt)

In [44]:
print(response)

\begin{comment}
According to the World Health Organisation (WHO), 91\% of the world's population reside in conditions where WHO's air quality guidelines levels were not met \cite{organizacion2021global}. This report from 2016 also indicated that ambient (outdoor) air pollution in both cities and rural areas was estimated to cause 4.2 million premature deaths worldwide. The research emphasized that implementing policies and investments to support cleaner transport, energy-efficient homes, power generation, industry, and better municipal waste management could significantly reduce outdoor air pollution. Another report estimates that air pollution globally accounts for approximately seven million premature deaths annually \cite{Gar21}, with the majority of these deaths being attributed to outdoor air pollution and the remainder largely due to poor indoor air quality from cooking practices. While most of these deaths occur in developing countries, with China and India contributing to about

In [20]:
import openai
import os
openai.api_key = os.environ["OPENAI_API_KEY"]
completion = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=[{'role': 'system', 'content': 'you are a helpful assistant'},
                {'role': 'user', 'content': prompt},
                {'role': 'assistant', 'content': response}, 
                {'role': 'user', 'content': "You loss some information! like 91%, WHO etc. You need to check whether the content you generated meets the requirements in the prompt, such as whether there is any loss of information, such as numbers, etc."}],
        temperature=0,
        response_format={"type": 'text'},
    )

In [21]:
completion.choices[0]["message"]["content"]

'\\section{Introduction}\nThe Urban Life and Air Pollution task at MediaEval 2022 required participants to predict the air quality index (AQI) value at +1, +5, and +7 days using an archive of air quality, weather data, and images from 16 CCTV cameras, with one image captured every 60 seconds \\cite{UA22}. This task necessitated that participating groups download the data from online sources for local processing. Gaps in air quality datasets are a widespread issue, with the challenge being more pronounced in poorer or developing countries \\cite{PINDER2019116794, Falge2001, Hui2004, Moffat2007, Kim2020}. In this paper, we delineate our approach to managing the extensive gaps in the air quality data that we encountered, ensuring the integrity of our analysis and findings.'

### Reflexion


You are an advanced academic polishing agent that can make the content more academically sound, clear, and engaging. 
You will be given a previous polishing trial in which you 根据一段review advise修改了a section (titled "{section_label}") of the paper in Latex format. 
这段文本的内容是
文本的结构是
修改意见是
修改要求是
对应的修改结果是.

你需要根据修改意见和修改要求反思你的修改结果，包括
(1) 修改结果是否满足修改要求
(2) 修改结果是否解决了修改意见中的问题。

Formatting Requirements:
你需要直接生成你的反思结果。如果你觉得已经满足了修改意见和修改要求，那么请返回None

Here are some examples:
{examples}

Reflection:

In [ ]:
reflect_prompt = """You are tasked with refining a section of an academic paper. A section (titled "{section_label}") of an academic paper in Latex format, has been previously revised based on a review advise.

The original section, logical structure, review advice, revise requirements for modification were provided. You made changes accordingly, resulting in a modified version of the text. 

Your task now is to reflect on the changes you've made. Consider the following points:

(1) Does the modified version meet the revise requirements specified for the changes?
(2) Does ithe modified versiont address the issues pointed out in the review advice?

Formatting Requirements:
Directly generate your reflection results. If you believe that your modifications have fully met the review advice and the specified revise requirements, please respond with "None".

Here are some examples for reference:
{examples}

The original section, logical structure, review advice, revise requirements and your modified version are as follows:

original section: {original_text}

logical structure of original text: {logical_structure}

review advice: {review_advise}

revise requirements: {revise_requirements}

modified version: {modified_text}

Please analyze your previous modification and provide your reflection.
"""

In [ ]:
reflect_prompt = """Objective: You are required to refine a section of an academic paper. This section, titled "{section_label}," has undergone previous revisions based on review advice. 
However, your current modifications have not fully met the review advice and revision requirements. 
The reasons for this shortfall will be provided to you. Your goal is to further modify the text to better align with the review advice and revision requirements. 
It's crucial to preserve the parts that don't need to be changed exactly as they are.

Formatting Requirements:
Present your output in Latex format aligned with provided section. You should directly output the revised text without anything additional.

Here are some examples for reference:
{examples}
(End of examples)

Components:

Original Section: {original_text}

Logical Structure of Original Text: {logical_structure}

Review Advice: {review_advise}

Revision Requirements: {revise_requirements}

Your Modified Version: {modified_text}

Unsatisfied Points of Modified Version: {unsatisfied_points}

Your Task: Analyze your previous modification and further revise your text. Address the unsatisfied points to better meet the review advice and revision requirements.
"""

In [ ]:
import numpy as np
img_path = './exp_result/our_method/DCU-AQ.tex_lan.pkl'
img_data = np.load(img_path, allow_pickle=True)
print(img_data)

In [7]:
print(len(img_data))

9


In [8]:
import joblib
ls_cache = joblib.load(img_path)

In [10]:
type(ls_cache)

list

In [60]:
from paper_class import *
paper = load_from_cache('./exp_result/our_method/moderately_poor.tex.pkl')

In [61]:
from polishGPT import polish

polish_level = "Test_structure"
features_p = ["更精确的措辞", "更具体的描述", "更连贯的表达", "更一致的风格", "更符合学术风格", "更正式的语法"]
length_req = None
length = None
paper_type = "会议"
paper_domain = "人工智能"
dt_res = {}
for key in paper.dt_section_content:
    input_text = paper.dt_section_content[key]
    res = polish(polish_level, features_p, length_req, length, input_text, paper_type, paper_domain, None)
    dt_res[key] = res
with open('./polishgpt/moderately.txt', 'w') as fp:
    fp.write(str(dt_res))

1
1
1
1
1
1
1
1
1


In [56]:
from gpt_academic import polish

dt_res = {}
for key in paper.dt_section_content:
    input_text = paper.dt_section_content[key]
    res = polish(input_text)
    dt_res[key] = res
with open('./gptacademic/extremely.txt', 'w') as fp:
    fp.write(str(dt_res))

In [62]:
prompt_3 = """I will provide you with three pieces of text. Please analyze them carefully and determine which one is better if you are a reviewer in Artificial Intelligence.

Below are some criteria you should take into consideration:

Conciseness: Removing unnecessary words to make the text clearer and more direct.
Coherence: Ensuring that sentences are well-organized and flow logically.
Academic Language: Using common academic terms and phrases (e.g., 'furthermore', 'thus').
Formal Grammar: Adhering to correct grammar and syntax, avoiding sentence fragments or off-topic sentences.
Logical Structure: Presenting clear and easily understandable logic, without causing confusion.

Here are the Three texts:

Text A:
{textA}
[End of text A]

Text B:
{textB}
[End of text B]

Text C:
{textC}
[End of text C]

Your task is to directly output your judgment, ranking Text A, Text B and Text C based on their goodness. No additional information is needed. For example, 
if you think Text A is the best, Text B is the second best and Text C is the worst, you should output "Text A, Text B and Text C".

Please analyze the texts based on these criteria and provide your judgment"""



In [45]:
from paper_class import *
import openai
from dotenv import load_dotenv
load_dotenv(dotenv_path = ".env")
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

res = {}
for paper in ['moderately_poor']:#, 'main', 'output']:
    for way in ['sec']:#, 'lan', 'stu']:
        paper_ours = load_from_cache(f'./exp_result/our_method/{paper}.tex.pkl')
        with open(f'./gptacademic/{paper.split('_')[0]}.txt', 'r') as fp:
            paper_academic = eval(fp.read())
        with open(f'./polishgpt/{paper.split('_')[0]}.txt', 'r') as fp:
            paper_polish = eval(fp.read())
        for key in paper_ours.dt_section_content:
            origin = paper_ours.dt_section_content[key]
            ours = paper_ours.dt_polishing_result[key]
            academic = paper_academic[key]
            polish = paper_polish[key]
            
            full_prompt = prompt_3.format(textA=academic, textB=ours, textC=polish)
            completion = openai.ChatCompletion.create(
                model="gpt-4-1106-preview",
                messages=[{'role': 'user', 'content': full_prompt}],
                temperature=0,
                response_format={"type": 'text'},
            )
            res[paper] = res.setdefault(paper, {})
            res[paper][way] = res[paper].setdefault(way, {})
            res[paper][way][key] = completion.choices[0]["message"]["content"]

In [46]:
for key1 in res:
    for key2 in res[key1]:
        for key3 in res[key1][key2]:
            print(f'{key1}_{key2}_{key3}: {res[key1][key2][key3]}')
            print('-+-'*20)

DCU-AQ_sec_Introduction: Text B, Text A, Text C
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Methodology: Text B, Text C, and Text A
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Data Analysis: Text B, Text C, Text A
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Image Processing: Text B, Text A, Text C
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Gap Filling: Text B, Text A, Text C
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Experiments: Text A, Text C, Text B
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Experimental Approach: Text B, Text A, Text C
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Results: Text A, Text B, Text C
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-
DCU-AQ_sec_Conclusions and Lessons Learned: Text A, Text B, Text C
-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-